In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from importlib import reload
from datetime import datetime

sys.path.insert(0, '../src')
import deepgeo.dataset.data_augment as dtaug
import deepgeo.dataset.utils as dsutils 
import deepgeo.common.geofunctions as gf
import deepgeo.networks.model_builder as mb

reload(dtaug)
reload(dsutils)
reload(mb)
reload(gf)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepgeo.common.geofunctions' from '../src/deepgeo/common/geofunctions.py'>

# Load input Dataset

In [2]:
# DATA_DIR = os.path.join(os.path.abspath(os.path.dirname('__file__')), '../', 'data_real', 'generated')
network = 'unet'
DATA_DIR = '/home/raian/doutorado/Dados/generated'
# DATASET_FILE = os.path.join(DATA_DIR, 'new_dataset_286x286_timesstack-2013-2017.npz')
DATASET = os.path.join(DATA_DIR, 'dataset_286x286_tmstk-2013-2017')

class_names = ['no_data', 'not_deforestation', 'deforestation']
# model_dir = os.path.join(DATA_DIR, 'tf_logs', network,
#                          'test_%s_%s' % (network, datetime.now().strftime('%d_%m_%Y-%H_%M_%S')))
model_dir = '/home/raian/doutorado/deepgeo/data_real/generated/tf_logs/test_debug'
# model_dir = os.path.join(DATA_DIR, 'tf_logs', 'experiments', 'unet', 'test_unet_29_04_2019-12_51_06')
train_tfrecord = os.path.join(DATASET, 'dataset_train.tfrecord')
test_tfrecord = os.path.join(DATASET, 'dataset_test.tfrecord')
val_tfrecord = os.path.join(DATASET, 'dataset_validation.tfrecord')

In [3]:
!rm -rf {model_dir}/*
!ls -al {model_dir}

total 8
drwxr-xr-x 2 raian digits 4096 May 22 13:21 .
drwxr-xr-x 3 raian digits 4096 Dec  7 19:52 ..


# Split dataset between train, test and validation data

In [4]:
# dataset = np.load(DATASET_FILE)

# print("Data Loaded:")
# print("  -> Images: ", len(dataset["images"]))
# print("  -> Labels: ", len(dataset["labels"]))
# print("  -> Classes: ", len(dataset["classes"]))

# print("Images shape: ", dataset["images"][0].shape, " - DType: ", dataset["images"][0].dtype)
# print("Labels shape: ", dataset["labels"][0].shape, " - DType: ", dataset["labels"][0].dtype)
# print('Classes: ', dataset['classes'])
# print("UNIQUE LABELS: ", np.unique(dataset["labels"]))

In [5]:
# train_images, test_images, valid_images, train_labels, test_labels, valid_labels = dsutils.split_dataset(dataset,
#                                                                                                          perc_test=20,
#                                                                                                          perc_val=20)

# print("Splitted dataset:")
# print("  -> Train images: ", train_images.shape)
# print("  -> Test images: ", test_images.shape)
# print("  -> Validation images: ", valid_images.shape)
# print("  -> Train Labels: ", train_labels.shape)
# print("  -> Test Labels: ", test_labels.shape)
# print("  -> Validation Labels: ", valid_labels.shape)
# total_train_chips = train_images.shape[0]

# Perform Data Augmentation

In [6]:
# angles = [90, 180, 270]
# rotated_imgs = dtaug.rotate_images(train_images, angles)
# flipped_imgs = dtaug.flip_images(train_images)

# train_images = np.concatenate((train_images, rotated_imgs, flipped_imgs))
# # train_images = np.concatenate((train_images, ))

# rotated_lbls = dtaug.rotate_images(train_labels, angles)
# flipped_lbls = dtaug.flip_images(train_labels)

# train_labels = np.concatenate((train_labels, rotated_lbls, flipped_lbls))
# # train_labels = np.concatenate((train_labels, )).astype(dtype=np.int32)

# print('Data Augmentation Applied:')
# print('  -> Train Images: ', train_images.shape)
# print('  -> Train Labels: ', train_labels.shape)

## Save to TFRecords

In [7]:
def print_progress(count, total):
    # Percentage completion.
    pct_complete = float(count) / total

    # Status-message.
    # Note the \r which means the line should overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [8]:
# def wrap_bytes(value):
#     return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def wrap_float(value):
#     return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# def wrap_int64(value):
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# def save_to_tfrecord(out_path, imgs, labels):
#     with tf.python_io.TFRecordWriter(out_path) as writer:
#         for i in range(imgs.shape[0]):
#             img = imgs[i, :, :, :]
#             lbl = labels[i, :, :, :]
            
#             height = img.shape[0]
#             width = img.shape[1]
#             channels = img.shape[2]
            
#             img_raw = img.tostring()
#             lbl_raw = lbl.tostring()
            
#             feature = {'image': wrap_bytes(img_raw),
#                        'label': wrap_bytes(lbl_raw),
#                        'channels': wrap_int64(channels),
#                        'height': wrap_int64(height),
#                        'width': wrap_int64(width)}
            
#             example = tf.train.Example(features=tf.train.Features(feature=feature))
#             writer.write(example.SerializeToString())

# save_to_tfrecord(train_tfrecord, train_images, train_labels)
# save_to_tfrecord(test_tfrecord, test_images, test_labels)
# save_to_tfrecord(val_tfrecord, valid_images, valid_labels)

## Computing weights

In [9]:
# def compute_weights_mean_proportion(batch_array, classes, classes_zero=['no_data']):
#     values, count = np.unique(batch_array, return_counts=True)
#     count = [count[i] if classes[i] not in classes_zero else 0 for i in range(0, len(count))]
#     total = sum(count)
#     proportions = [i / total for i in count]
#     mean_prop = sum(proportions)/ (len(proportions) - len(classes_zero))
#     weights = [mean_prop / i if i != 0 else 0 for i in proportions]
#     return weights


# weights_train = compute_weights_mean_proportion(train_labels, dataset['classes'])
# weights_eval = compute_weights_mean_proportion(test_labels, dataset['classes'])

In [11]:
def parse(serialized):
    features = {'label': tf.FixedLenFeature([], tf.string, default_value=''),
                'height': tf.FixedLenFeature([], tf.int64, default_value=0),
                'width': tf.FixedLenFeature([], tf.int64, default_value=0)}

    parsed_features = tf.parse_single_example(serialized=serialized, features=features)
    height = parsed_features['height']
    width = parsed_features['width']

    label = tf.decode_raw(parsed_features['label'], tf.int32)
    label = tf.reshape(label, [height, width, 1])

    return label

def compute_weights_mean_proportion(tfrecord, classes, classes_zero=['no_data']):
    tf.enable_eager_execution()
    train_ds = tf.data.TFRecordDataset(tfrecord)
    train_ds = train_ds.map(parse)
    tot_count = [0] * len(classes)
    for label in train_ds:
        label = label.numpy()
        unique, count = np.unique(label, return_counts=True)
        for k, v in enumerate(unique):
            if classes[v] not in classes_zero:
                tot_count[v] += count[k]
    total = sum(tot_count)
    proportions = [i / total for i in tot_count]
    mean_prop = sum(proportions)/ (len(proportions) - len(classes_zero))
    weights = [mean_prop / i if i != 0 else 0 for i in proportions]
    return weights
    
weights_train = compute_weights_mean_proportion(train_tfrecord, class_names, ['no_data'])
weights_eval = compute_weights_mean_proportion(test_tfrecord, class_names, ['no_data'])

Parsing " /home/raian/doutorado/Dados/generated/dataset_286x286_tmstk-2013-2017/dataset_train.tfrecord "
Parsing " /home/raian/doutorado/Dados/generated/dataset_286x286_tmstk-2013-2017/dataset_test.tfrecord "


In [12]:
# print(weights_train)
# print(weights_eval)

[0, 0.5028229179643342, 89.06084489828929]
[0, 0.5028547403476658, 88.07363877398384]


# Train the Network

In [13]:
params = {
    'epochs': 1,
    'batch_size': 40,
    'chip_size': 286,
    'bands': 10,
    'filter_reduction': 0.5,
    'learning_rate': 0.1,
    'learning_rate_decay': True,
    'decay_rate': 0.95,
    'l2_reg_rate': 0.0005,
    # 'var_scale_factor': 2.0,  # TODO: Put the initializer as parameter
    'chips_tensorboard': 2,
    # 'dropout_rate': 0.5,  # TODO: Put a bool parameter to apply or not Dropout
    'fusion': 'early',
    'loss_func': 'weighted_crossentropy',
    'data_aug_ops': ['rot90', 'rot180', 'rot270', 'flip_left_right',
                     'flip_up_down', 'flip_transpose'],
    'class_weights': {'train': weights_train, 'eval': weights_eval},
    'num_classes': len(class_names),
    'class_names': ['no data', 'not deforestation', 'deforestation'],
    'num_compositions': 2,
    'bands_plot': [[1, 2, 3], [6, 7, 8]],
    'Notes': 'Testing.'
}
reload(mb)
model = mb.ModelBuilder(network)

In [14]:
model.train(train_tfrecord, test_tfrecord, params, model_dir)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not usin

INFO:tensorflow:Finalize strategy.
INFO:tensorflow:Loss for final step: 0.28406376.


In [15]:
model.validate(valid_images, valid_labels, params, model_dir)

NameError: name 'valid_images' is not defined